## Quickstart
- Compare results of the runs
- Choose best run and register as a model
- Deploy the model to a rest API
- Build container image

In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from mlflow.models import infer_signature

In [3]:
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep=";")
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [5]:
# Split the data into training, validation and test
train,test = train_test_split(data,test_size=0.25,random_state=42)

X_train = train.drop(['quality'],axis=1).values
# Ravel is for getting it as a 1D array
y_train = train['quality'].values.ravel()

# Test dataset
X_test = test.drop(['quality'],axis=1).values
y_test = test['quality'].values.ravel()

# Further split the data to get the validation set
X_train,X_valid,y_train,y_valid = train_test_split(X_train,y_train,test_size=0.2,random_state=42)

signature = infer_signature(X_train,y_train)

In [7]:
import mlflow

In [21]:
# ANN Model
def train_model(params,epochs,X_train,y_train,X_valid,y_valid,X_test,y_test):
    # Model arquitecture

    # Normalize data getting average of the column
    mean = np.mean(X_train,axis=0)
    var = np.var(X_train,axis=0)

    # Create model
    model = keras.Sequential(
        [
        keras.Input([X_train.shape[1]]),
        keras.layers.Normalization(mean = mean,variance=var),
        keras.layers.Dense(64,activation = 'relu'),
        keras.layers.Dense(1)
        ]

    )

    # Compile the model
    model.compile(optimizer = keras.optimizers.SGD(learning_rate=params['lr'],
                                                   momentum = params['momentum']
                                                   ), 
                                                   loss='mean_squared_error',
                                                   metrics = [keras.metrics.RootMeanSquaredError()])
    

    # Train the model with lr and momentum with MLFlow tracking

    with mlflow.start_run(nested = True):
        model.fit(X_train,y_train,validation_data=(X_valid,y_valid),
                  epochs = epochs, batch_size = 64)
        
        # Evalute the model

        eval_result = model.evaluate(X_valid,y_valid,batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        # Log the model
        mlflow.tensorflow.log_model(model,"model",signature=signature)


        return {'loss':eval_rmse,"status":STATUS_OK,"model":model}

In [22]:
def objective(params):
    # Track parameters result from each function
    result = train_model(params,
                         epochs=3,
                         X_train = X_train,
                         y_train= y_train,
                         X_valid= X_valid,
                         y_valid= y_valid,
                         X_test= X_test,                       
                         y_test= y_test)
    
    return result

In [23]:
space = {
    "lr" : hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum" : hp.uniform("momentum",0.0,1.0)
}

In [24]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct hyperparamters search
    trials = Trials()
    best = fmin(space=space,
                fn=objective,
                algo=tpe.suggest,
                max_evals=4,
                trials = trials)
    best_run = sorted(trials.results,key=lambda x:x['loss'])[0]

    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse",best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"],"model",signature=signature)

    print("Best parameters",best)
    print("Best eval rmse",best_run["loss"])

Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 33s 752ms/step - loss: 36.0813 - root_mean_squared_error: 6.0068
18/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 32.3778 - root_mean_squared_error: 5.6883   
36/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 29.9623 - root_mean_squared_error: 5.4678
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 23.6131 - root_mean_squared_error: 4.8593 - val_loss: 14.6584 - val_root_mean_squared_error: 3.8286

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 14.2517 - root_mean_squared_error: 3.7751
18/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 13.4563 - root_mean_squared_error: 3.6673 
36/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12.4119 - root_mean_squared_error: 3.5190
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 9.7057 - root_mean_squared_error: 3.1154 - val_loss: 6.0440 - val_root_mean_squared_error: 2.4584

Epoch 3/3                                            

2025/08/28 11:52:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 28s 634ms/step - loss: 38.3189 - root_mean_squared_error: 6.1902
22/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 32.4504 - root_mean_squared_error: 5.6914   
35/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 30.1586 - root_mean_squared_error: 5.4811
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 21.6868 - root_mean_squared_error: 4.6569 - val_loss: 11.0660 - val_root_mean_squared_error: 3.3266

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 9.9953 - root_mean_squared_error: 3.1615
13/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 10.1971 - root_mean_squared_error: 3.1930
30/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9.2489 - root_mean_squared_error: 3.0371 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 6.6906 - root_mean_squared_error: 2.5866 - val_loss: 4.1002 - val_root_mean_squared_error: 2.0249

Epo

2025/08/28 11:52:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 36s 822ms/step - loss: 36.0587 - root_mean_squared_error: 6.0049
20/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 32.6764 - root_mean_squared_error: 5.7134   
43/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 29.0940 - root_mean_squared_error: 5.3823
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 22.6786 - root_mean_squared_error: 4.7622 - val_loss: 12.2176 - val_root_mean_squared_error: 3.4954

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 11.2338 - root_mean_squared_error: 3.3517
19/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 10.8009 - root_mean_squared_error: 3.2856 
36/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.9896 - root_mean_squared_error: 3.1569 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 7.7736 - root_mean_squared_error: 2.7881 - val_loss: 4.4583 - val_root_mean_squared_error: 2.1115

E

...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 30s 669ms/step - loss: 41.2181 - root_mean_squared_error: 6.4201
17/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 36.3724 - root_mean_squared_error: 6.0279   
29/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 34.0314 - root_mean_squared_error: 5.8264
41/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 31.9882 - root_mean_squared_error: 5.6431
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 24.4168 - root_mean_squared_error: 4.9413 - val_loss: 13.4503 - val_root_mean_squared_error: 3.6675

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - loss: 12.5080 - root_mean_squared_error: 3.5367
16/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12.1968 - root_mean_squared_error: 3.4919 
32/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 11.2969 - root_mean_squared_error: 3.3577
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 8.4142 - ro

2025/08/28 11:53:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [01:31<00:00, 22.87s/trial, best loss: 1.6304324865341187]


2025/08/28 11:53:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Best parameters {'lr': np.float64(0.0009234540504826522), 'momentum': np.float64(0.20993046825695272)}
Best eval rmse 1.6304324865341187


## Inferencing